In [ ]:
%pip install -q numpy
%pip install -q scipy
%pip install -q matplotlib
%pip install -q scikit-learn
%pip install -q pandas

In [ ]:
import numpy as np
from numpy import ndarray

import scipy as sp
from scipy import stats

import matplotlib.pyplot as plt

import pandas as pd
from pandas import DataFrame

In [ ]:
from os import getcwd

CWD: str = getcwd()
ASSETS_DIR: str = CWD + "/Assets-2_2"
DATASET_DIR: str = ASSETS_DIR + "/heightWeight.csv"

# Questão A

In [ ]:
def linear_function(x: ndarray) -> ndarray:
    return x * 2 + 1

In [ ]:
def uniform_distribution(x: ndarray) -> ndarray:
    return np.random.uniform(size=len(x))

In [ ]:
X_POINTS: ndarray = np.linspace(0, 1, 9)

In [ ]:
fig, axes = plt.subplots(1, 2, figsize=(12, 6))
ax1, ax2 = axes

# ---- Linear function ----
y1: ndarray = linear_function(X_POINTS)
ax1.scatter(X_POINTS, y1)

# get the linear regression
slope1, intercept1, r_value1, p_value1, std_err1 = stats.linregress(X_POINTS, y1)
line1: ndarray = slope1 * X_POINTS + intercept1
ax1.plot(X_POINTS, line1, label="Linear regression", linestyle="--")

# ---- Uniform distribution ----
y2: ndarray = uniform_distribution(X_POINTS)
ax2.scatter(X_POINTS, y2)

# get the linear regression
slope2, intercept2, r_value2, p_value2, std_err2 = stats.linregress(X_POINTS, y2)
line2: ndarray = slope2 * X_POINTS + intercept2
ax2.plot(X_POINTS, line2, label="Linear regression", linestyle="--")

ax1.set_title("Linear function")
ax2.set_title("Uniform Distribution")

plt.plot()

# Questão B

## Valores para a visualização dos dados

In [ ]:
BINS: int = 40
DENSITY: bool = True

## Carregando o dataset de pesos e alturas do [Kaggle](https://www.kaggle.com/datasets/burnoutminer/heights-and-weights-dataset?resource=download)

Esse dataset contém o total de 25 mil amostras de diferentes humanos de 18 anos de idades. Suas colunas são `Height(Inches)` em polegadas (_inches_) e `Weight(Pounds)` em libras (_pounds_). Utilizaremos apenas a coluna de altura (`Height(Inches)`) em passaremos ela para centímetros para melhorar a visualização

In [ ]:
df: DataFrame = pd.read_csv(DATASET_DIR)

display(df)

## Manipulação dos dados

Iremos transformar a coluna `Height(Inches)` em valores mais comuns para o brasileiro, então converteremos os valores para centímetros e iremos trocar o nome da coluna.

Para conversão, basta multiplicar a quantidade de polegadas vezes 2.54. Também converteremos a coluna de `weight` para que ela esteja convertida para quilogramas, basta 
multiplicar cada amostra por 0.453592.

In [ ]:
HEIGHT_COLUMN: str = "height"
WEIGHT_COLUMN: str = "weight"

# create a copy, rename the Height(Inches) to height, return just the height column
df_cleaned: DataFrame = df.copy().rename(columns={"Height(Inches)":HEIGHT_COLUMN, "Weight(Pounds)":WEIGHT_COLUMN}).drop(columns=["Index"])

# convert to cm
df_cleaned[HEIGHT_COLUMN] = df_cleaned[HEIGHT_COLUMN].map(lambda x: x*2.54)

# convert to kg
df_cleaned[WEIGHT_COLUMN] = df_cleaned[WEIGHT_COLUMN].map(lambda x: x*0.453592)

display(df_cleaned)

## Análise da altura

Calculamos a média e o desvio padrão para conseguirmos encontrar o intervalo de confiança da distribuição normal. Para buscar esse intervalo 
utilizamos o `scipy.stats.norm.interval` para pegarmos exatamente o intervalo que desejamos.

A formula usada é:

$$
\text{CI} = \bar{x} \pm z \times \frac{\sigma}{\sqrt{n}}
$$

- $z$ - Nível de confiança
- $\bar{x}$ - Média da amostra
- $\sigma$ - Desvio padrão da amostra
- $n$ - Tamanho da amostra

In [ ]:
mean: float = df_cleaned[HEIGHT_COLUMN].mean()
std: float = df_cleaned[HEIGHT_COLUMN].std() # type: ignore

confidences: tuple[float, ...] = (0.85, 0.9, 0.95, 0.99)
results: dict[float, tuple] = {}

for confidence in confidences:
    interval: tuple = stats.norm.interval(confidence, loc=mean, scale=std)
    print(f"Para uma confiança de {int(confidence * 100)}%, temos o intervalo: [{interval[0]}, {interval[1]}]")
    results[confidence] = interval

## Criando o histograma para mostrar graficamente a distribuição

In [ ]:
colors: tuple[str, str, str, str] = ('#66b3ff', '#3399ff', '#0066cc', '#0044cc')

plt.figure(figsize=(10, 6))

# histogram
plt.hist(df_cleaned[HEIGHT_COLUMN], density=DENSITY, bins=BINS, alpha=0.3, color='grey', label="Dados (Amostra)")

# mean line
plt.axvline(mean, color="red", linestyle="--", label=f"Média: {mean:.5f}")

# confidence bar
y_max: float = plt.gca().get_ylim()[1] # graphic top
confidence_heights: tuple[float, float, float, float] = (y_max*0.1, y_max*0.2, y_max*0.3, y_max*0.4)

for i, (confidence, interval) in enumerate(results.items()):
    erro = (interval[1] - interval[0]) / 2

    plt.errorbar(mean, confidence_heights[i], xerr=erro, fmt="o", capsize=8, color=colors[i], label=f"Intervalo de Confiança: {int(confidence*100)}%")

plt.ylim(0, y_max * 1.2) # increase the graphic height
plt.title("Distribuição da Altura e Intervalo de Confiança")
plt.xlabel("Altura (cm)")
plt.ylabel("Densidade (número de amostras no bin)")
plt.legend()
plt.show()

# Questão C

## Definindo a quantidade de amostras para cada grupo

In [ ]:
SAMPLES_NUMBER: int = 100

## Criando os grupos com base no dataset da questão anterior

In [ ]:
generator = np.random.default_rng()

# get a random sample of values to control group
control = df_cleaned[HEIGHT_COLUMN].sample(SAMPLES_NUMBER, random_state=generator)

# get a random sample of values to test group
test = df_cleaned[HEIGHT_COLUMN].sample(SAMPLES_NUMBER, random_state=generator)

## Utilizando `scipy.stats.ttest_ind` para verificar o p_value e o t_stat, ambos provindos da fórmula estatística da distribuição T-Student

In [ ]:
t_stat, p_value = stats.ttest_ind(control, test)

print(f"Média Controle: {control.mean()}")
print(f"Média Teste: {test.mean()}")
print(f"Estatística T: {t_stat}")
print(f"P-value: {p_value}")

## Avaliação do p_value

Se o valor de p_value for menor que um dado $\alpha$ qualquer no intervalo $[0, 1]$, então a nossa hipótese nula está correta. Quantos mais valores aleatórios forem buscados, menor 
será o t_stat, o qual representa a diferença entre o grupo controle e o grupo de teste. Isso significa que mais dados dão uma confiança muito maior para os nossos resultados. Dado essa
explicação, decidi utilizar um valor de $\alpha$ igual a 0.05, para que o `p_value` seja menor que ele, ou seja, uma confiança maior que 95%.

Logo, a Hipótese Nula ($H_0$) é de que os dados que pegamos não são distintos. E a Hipótese Alternativa ($H_1$) é de que os dados são estatisticamente diferentes entre os grupos, ou 
seja, são diferentes comuns.

- $H_0$ - Os grupos são estatisticamente iguais
- $H_1$ - Os grupos são estatisticamente diferentes

Em outras palavras, o $\alpha$ é um valor de confiança que aceitamos tolerar. Caso seja um $\alpha$ muito grande, então a chance de ser por a caso é maior e, por isso, o resultado não
é considerado confiável. Desse modo, o `p_value` reflete a chance de termos conseguido esse resultado por a caso.

In [ ]:
alpha: float = 0.05

if p_value < alpha: # type: ignore
    print(f"Decisão: Hipótese nula rejeitada. O p-value é suficientemente confiável: {p_value}")
    print(f"Conclusão: Existe uma diferença estatisticamente significativa entre os grupos")
else:
    print(f"Decisão: Hipótese nula aceita. O p-value não é suficientemente confiável: {p_value}")
    print(f"Conclusão: Existe uma diferença estatisticamente pequena entre os grupos")

# Questão D

O slide pode ser encontrado no [link](https://docs.google.com/presentation/d/18qr6W9VnDn1slNsL1n7vQRvynUyGTF1Agm1K0FSbRpo/edit?usp=sharing)

# Questão E

## Calculando o valor de $t$ crítico para utilizar com a distribuição _t-student_

In [ ]:
from scipy.stats import t

confidence_level: float = 0.975
degrees_of_freedon: int = 49

t_critical: ndarray = t.ppf(confidence_level, degrees_of_freedon)
print(t_critical)

## Calculando o intervalo de confiança com base da fórmula

$$
\bar{x} \pm t^* \cdot \frac{s}{\sqrt{n}}
$$

- $\bar{x}$ - Média dos dados coletados
- $t^*$ - Valor dado pela distribuição _t-student_
- $s$ - Desvio padrão das amostras
- $n$ - Número de amostras

Por meio dessa fórmula é possível identificar qual é a chance o intervalo de confiança

In [ ]:
mean: float = 75.9

# Questão F

# Questão G

# Questão H

# Questão I